In [15]:
import os
import sys
import pandas as pd
import numpy as np
script_dir = os.getcwd()
file = 'simulator1.csv'
data = pd.read_csv(file)
data.head()
x_label = ["Azimuth (deg)" , "Elevation (deg)" , "EIRP (dBW)" , "Xmtr Gain (dB)" , "Xmtr EIRP Intensity (dBW/Sterad)" , "Atmos Loss (dB)"]
y_label = ["Rain Loss (dB)" , "CloudsFog Loss (dB)"]


def extract_data(i, j):
    X = []
    Y = []
    for i in range(i, j):
        x_sample = []
        y_sample = []
        for label in x_label:
            x_sample.append(float((data[label][i])))
        X.append(x_sample)
        for label in y_label:
            y_sample.append(float((data[label][i])))
        Y.append(y_sample)
    return np.asarray(X), np.asarray(Y)

x_train, y_train =  extract_data(0, 590)
x_test, y_test = extract_data(595, 1180)
x_validation, y_validation = extract_data(1185, 1691)

In [16]:
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))


In [17]:
import pandas
pandas.__version__
import numpy
#from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers.convolutional import Conv1D    
from numpy import concatenate


# fix random seed for reproducibility
numpy.random.seed(7)

In [18]:
# design network
model = Sequential()
model.add(LSTM(len(x_label), input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(2))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(x_train, y_train, epochs=50, batch_size=72, validation_data=(x_test, y_test), verbose=2, shuffle=False)
"""
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
"""

Train on 590 samples, validate on 585 samples
Epoch 1/50
 - 0s - loss: 0.2310 - val_loss: 0.2099
Epoch 2/50
 - 0s - loss: 0.2020 - val_loss: 0.1807
Epoch 3/50
 - 0s - loss: 0.1758 - val_loss: 0.1530
Epoch 4/50
 - 0s - loss: 0.1573 - val_loss: 0.1314
Epoch 5/50
 - 0s - loss: 0.1436 - val_loss: 0.1168
Epoch 6/50
 - 0s - loss: 0.1270 - val_loss: 0.1046
Epoch 7/50
 - 0s - loss: 0.1096 - val_loss: 0.0951
Epoch 8/50
 - 0s - loss: 0.0956 - val_loss: 0.0872
Epoch 9/50
 - 0s - loss: 0.0833 - val_loss: 0.0759
Epoch 10/50
 - 0s - loss: 0.0730 - val_loss: 0.0656
Epoch 11/50
 - 0s - loss: 0.0643 - val_loss: 0.0571
Epoch 12/50
 - 0s - loss: 0.0564 - val_loss: 0.0487
Epoch 13/50
 - 0s - loss: 0.0497 - val_loss: 0.0438
Epoch 14/50
 - 0s - loss: 0.0466 - val_loss: 0.0393
Epoch 15/50
 - 0s - loss: 0.0434 - val_loss: 0.0345
Epoch 16/50
 - 0s - loss: 0.0404 - val_loss: 0.0302
Epoch 17/50
 - 0s - loss: 0.0370 - val_loss: 0.0306
Epoch 18/50
 - 0s - loss: 0.0351 - val_loss: 0.0295
Epoch 19/50
 - 0s - loss: 0

"\n# plot history\npyplot.plot(history.history['loss'], label='train')\npyplot.plot(history.history['val_loss'], label='test')\npyplot.legend()\npyplot.show()\n"

In [19]:
#model.predict(x_test)

In [23]:
# make a prediction
yhat = model.predict(x_test)
print(yhat)
print("y_test", y_test)
print(len(yhat),len(y_test))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, x_test[:, 1:]), axis=1)
# inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
y_test = y_test.reshape((len(y_test), 1))
inv_y = concatenate((y_test, x_test[:, 1:]), axis=1)
# inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

[[0.00055302 0.00093309]
 [0.00055302 0.00093309]
 [0.00055302 0.00093309]
 ...
 [0.00055302 0.00093309]
 [0.00055302 0.00093309]
 [0.00055302 0.00093309]]
y_test [[0.0024 0.0029]
 [0.0024 0.0029]
 [0.0023 0.0028]
 ...
 [0.0024 0.0029]
 [0.0024 0.0029]
 [0.0024 0.0029]]
585 585


"x_test = x_test.reshape((x_test.shape[0], x_test.shape[2]))\n# invert scaling for forecast\ninv_yhat = concatenate((yhat, x_test[:, 1:]), axis=1)\n# inv_yhat = scaler.inverse_transform(inv_yhat)\ninv_yhat = inv_yhat[:,0]\n# invert scaling for actual\ny_test = y_test.reshape((y_test.shape[0], y_test.shape[2]))\n#y_test = y_test.reshape((len(y_test), 1))\ninv_y = concatenate((y_test, x_test[:, 1:]), axis=1)\n# inv_y = scaler.inverse_transform(inv_y)\ninv_y = inv_y[:,0]\n# calculate RMSE\nrmse = sqrt(mean_squared_error(inv_y, inv_yhat))\nprint('Test RMSE: %.3f' % rmse)"